In [59]:
import pandas as pd
import numpy as np
import glob
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [60]:
ruta = '/content/drive/MyDrive/VentaPorMeses'

archivos = os.listdir(ruta)
archivos

dataframes = {}

for archivo in archivos:
  if archivo.endswith('.csv'): # Asegurarse que solo se consideren archivos csv
    mes = archivo.split('_')[3].split('.')[0]   # Separa en una lista el nombre de cada csv, para tomar el mes
    ruta_completa = os.path.join(ruta, archivo) # Añade la ruta de cada dataset
    df = pd.read_csv(ruta_completa) # Cargar el csv en un Dataframe
    df_ventas = df.copy()

    # Transformo todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

    # Elimino todos los Nan y me quedo solo con los valores numericos
    df_ventas = df_ventas.dropna()

    # Transformo todos los valores numericos al tipo de dato int
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)

    dataframes[mes] = df_ventas   # con el mes agrega su respectivo dataset en un diccionario, queda como clave cada mes y valor el dataset


<ipython-input-60-a7f6c74b93a3>:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-60-a7f6c74b93a3>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Funciones para extraccion de datos

In [61]:
def extraer_datos(datos,tipo): #El parametro tipo sirve para identificar el tipo de dato que se quiere extraer
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  datos['Mes'] = datos['Fecha de Pedido'].dt.month
  if(tipo =="ciudad"):
    datos[['Calle','Ciudad','PO BOX']] = datos['Dirección de Envio'].str.split(', ',expand=True)
    # Separa el codigo postal
    datos[['Estado', 'Cod Postal']] = datos['PO BOX'].str.split(' ',expand=True)
  if(tipo =="dia_semana"):
    datos['Dia de la Semana'] = datos['Fecha de Pedido'].dt.day_name()

  return datos

###Punto 1 y 2

####1. Comportamiento de las ventas en los distintos meses:

#####Variaciones de las ventas a lo largo de los meses

In [62]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
totales_ventas = []
nombres_meses = []

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes,"")
  total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
  totales_ventas.append(total_ventas_mes)
  nombres_meses.append(mes)


Grafico

In [63]:
# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',
))
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)
fig.show()

#####Mes con mas ventas

In [64]:
max_ventas = max(totales_ventas)
mes_max_ventas = nombres_meses[totales_ventas.index(max_ventas)]
print("El mes donde se realizaron mas ventas fue",mes_max_ventas)

El mes donde se realizaron mas ventas fue Diciembre


#####Ingresos totales por mes

In [65]:
# Calcular la ganancia por mes
ganancia_mensual = []
for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes["Ganancia"] = dataframe_mes["Precio Unitario"] * dataframe_mes["Cantidad Pedida"]
  ganancia_mensual.append(round(dataframe_mes["Ganancia"].sum(), 2))


Grafico

In [66]:
fig = go.Figure(data=[go.Bar(x=meses, y=ganancia_mensual,marker_color="green")])

# Personalizar el diseño del gráfico
fig.update_layout(
    title="Ganancia Mensual",
    xaxis_title="Mes",
    yaxis_title="Ganancia $",
)
fig.show()

####2. Optimización de la publicidad y patrón de ventas por hora:

#####Variacion de ventas y ganancias segun las horas

In [67]:
resultados_por_mes = []
for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes,"")
  ventas_por_hora = dataframe_mes.groupby(["Mes", "Hora"]).size().reset_index(name="Cantidad de Ventas")

  # Agregar los resultados al listado
  resultados_por_mes.append(ventas_por_hora)


Grafico

In [68]:
# Crear un DataFrame para almacenar los datos de ventas por hora
df_ventas_por_hora = pd.DataFrame()

for idx, mes in enumerate(meses):
    ventas_por_hora = resultados_por_mes[idx]
    ventas_por_hora["Mes"] = mes  # Agregar una columna para el mes
    df_ventas_por_hora = df_ventas_por_hora.append(ventas_por_hora, ignore_index=True)

# Obtener las horas del día
horas_del_dia = df_ventas_por_hora["Hora"].unique()

# Crear un DataFrame con las horas como columnas y los meses como índice
df_ventas_mensual = pd.DataFrame(columns=horas_del_dia, index=meses)
for idx, mes in enumerate(meses):
    data_mes = df_ventas_por_hora[df_ventas_por_hora["Mes"] == mes]
    for hora in horas_del_dia:
        data_hora = data_mes[data_mes["Hora"] == hora]
        total_ventas = data_hora["Cantidad de Ventas"].sum()
        df_ventas_mensual.at[mes, hora] = total_ventas

# Convertir las columnas de horas a tipo numérico
df_ventas_mensual = df_ventas_mensual.apply(pd.to_numeric)

# Crear el gráfico de barras
fig = go.Figure()

for hora in horas_del_dia:
    fig.add_trace(go.Bar(
        x=meses,  # Meses en el eje X
        y=df_ventas_mensual[hora],
        name=f'Hora {hora}',
    ))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Cantidad de Pedidos por Hora y Mes',
    xaxis_title='Mes',
    yaxis_title='Cantidad de Pedidos',
    barmode='group',  # Barras apiladas
)

# Mostrar el gráfico
fig.show()

<ipython-input-68-3184bba4ed84>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



#####Hora de mayor actividad de cada mes

In [69]:
horas_mas_activas = df_ventas_mensual.idxmax(axis=1)

Grafico

In [70]:
import plotly.express as px

# Crear un DataFrame con los meses y las horas de máxima actividad
df_horas_actividad = pd.DataFrame({'Mes': meses, 'Hora pico': horas_mas_activas})

# Crear el gráfico de dispersión
fig = px.scatter(df_horas_actividad, x='Mes', y='Hora pico', title='Hora pico de cada Mes')

# Agregar las etiquetas de texto (horas) arriba de los puntos
for i, row in df_horas_actividad.iterrows():
    fig.add_annotation(
        text=row['Hora pico'],
        x=row['Mes'],
        y=row['Hora pico'] + 1,  # Aumentar la coordenada Y
        showarrow=False,
        font=dict(size=12),
    )

# Personalizar el diseño del gráfico
fig.update_layout(
    xaxis_title='Mes',
    yaxis_title='Hora pico',
)

# Mostrar el gráfico
fig.show()

###Punto 3 y 4


####3. Distribución de ventas por ubicación:

#####¿En qué ciudades se han registrado las mayores ventas?

In [71]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

df_agrupado = pd.DataFrame()

for mes in meses:
  df = dataframes[mes]
  df = extraer_datos(df,"ciudad")
  df_agrupado = df_agrupado.append(df)

<ipython-input-71-d6f78fd708d8>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-71-d6f78fd708d8>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-71-d6f78fd708d8>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-71-d6f78fd708d8>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-71-d6f78fd708d8>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-71-d6f78fd708d8>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pand

In [72]:
ventas_por_ciudad = df_agrupado.groupby('Ciudad')['Cantidad Pedida'].sum()

fig = px.bar(ventas_por_ciudad,
             x='Cantidad Pedida',  # Las cantidades pedidas en el eje x
             y=ventas_por_ciudad.index,  # Las ciudades en el eje y
             title='Ventas por Ciudad',
             text='Cantidad Pedida',  # Etiquetas en las barras
             color='Cantidad Pedida',  # Colorear las barras por cantidad pedida
             color_continuous_scale='viridis',  # Paleta de colores
             width=800, height=500, # Tamaño del gráfico
             category_orders={'Ciudad': ventas_por_ciudad.index})  # Ordenar las ciudades

fig.update_xaxes(title_text='Cantidad Pedida')
fig.update_yaxes(title_text='Ciudad')
fig.show()

#####¿Cómo se comparan las ventas en diferentes estados o regiones?

In [73]:
# Uso el dataframe agrupado anterior y solo le cambio los parametros

ventas_por_estado = df_agrupado.groupby('Estado')['Cantidad Pedida'].sum()

fig = px.bar(ventas_por_estado,
             x='Cantidad Pedida',  # Las cantidades pedidas en el eje x
             y=ventas_por_estado.index,  # Los estados en el eje y
             title='Ventas por Estado',
             text='Cantidad Pedida',  # Etiquetas en las barras
             color='Cantidad Pedida',  # Colorear las barras por cantidad pedida
             width=800, height=500, # Tamaño del gráfico
             color_continuous_scale='viridis',  # Paleta de colores
             category_orders={'Estado': ventas_por_estado.index},  # Ordenar los estados
             )

fig.update_xaxes(title_text='Cantidad Pedida')
fig.update_yaxes(title_text='Estado')
fig.show()

#####¿Existe variación en las ventas por estado a lo largo de los meses?

In [74]:
fig = go.Figure()

# Definir el tamaño de la figura (ancho x alto)
fig.update_layout(width=800, height=500)

for estado in df['Estado'].unique():
    # Creo una traza para cada estado
    ventas_estado_meses = []

    for mes in meses:
        df = dataframes[mes]
        df = extraer_datos(df,"ciudad")

        ventas_estado = df[df['Estado'] == estado]['Cantidad Pedida'].sum()
        ventas_estado_meses.append(ventas_estado)

    # Agrego un gráfico de dispersión para las ventas del estado a lo largo de los meses
    fig.add_trace(go.Scatter(
        x=meses,
        y=ventas_estado_meses,
        line=dict(shape='spline', smoothing=1.0, width=2),  # Reducir el grosor de la línea y suavizado
        mode='lines+markers',
        name=estado
    ))

# Configuro los ejes y el título
fig.update_layout(
    title="Ventas de cada Estado a lo largo de los Meses",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Cantidad Pedida"),
)

fig.show()


#### 4. Análisis del producto más vendido:

##### ¿Cuál es el producto más vendido en general y en cada mes?

In [75]:
# Crear un gráfico de barras para mostrar las cantidades vendidas por producto en general
ventas_por_producto = df_agrupado.groupby('Producto')['Cantidad Pedida'].sum()

fig_general = px.bar(ventas_por_producto,
                      x=ventas_por_producto.index,
                      y='Cantidad Pedida',
                      color='Cantidad Pedida',
                      title='Ventas por Producto (General)',
                      text='Cantidad Pedida',  # Agregar etiquetas en las barras
                      color_continuous_scale='viridis',  # Paleta de colores
                      category_orders={'Producto': ventas_por_producto.sort_values(ascending=False).index.tolist()},  # Corregir el orden
                      )

# Cambiar la orientación de las etiquetas en el eje x
fig_general.update_xaxes(title_text='Producto', tickangle=-45)  # Cambiar el ángulo de las etiquetas

# Ajustar el tamaño de la figura
fig_general.update_layout(width=800, height=400)

# Mostrar valores en formato separador de miles
fig_general.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig_general.update_yaxes(title_text='Cantidad Pedida')
fig_general.show()


In [76]:
# Crear un objeto figura
fig = go.Figure()

# Crear una traza para cada producto
for producto in ventas_por_producto.index.unique():
    ventas_producto_meses = []  # Lista para almacenar las ventas del producto en cada mes

    for mes in meses:
        df = dataframes[mes]
        df = extraer_datos(df,"ciudad")  # Aplicar la función de extracción de datos si es necesario

        ventas_producto = df[df['Producto'] == producto]['Cantidad Pedida'].sum()
        ventas_producto_meses.append(ventas_producto)

    # Agregar un gráfico de líneas con suavizado para las ventas del producto a lo largo de los meses
    fig.add_trace(go.Scatter(
        x=meses,
        y=ventas_producto_meses,
        mode='lines+markers',
        line=dict(shape='spline', smoothing=1.0, width=2),  # Reducir el grosor de la línea y suavizado
        name=producto
    ))

# Configurar los ejes y el título
fig.update_layout(
    title="Ventas de cada Producto a lo largo de los Meses",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Cantidad Pedida"),
)

# Definir el tamaño de la figura (ancho x alto)
fig.update_layout(width=800, height=500)

# Mostrar el gráfico
fig.show()

###Punto 5 y 6

In [77]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

In [78]:
dataframes_copia = dataframes
for mes in meses:
    dataframe_mes = dataframes_copia[mes]
    dataframes_copia[mes] = extraer_datos(dataframe_mes,"")

####Punto 5

In [79]:
dias = list(range(1, 32))

# Creo una matriz de datos para las ventas por día en los diferentes meses
data = []

for mes in meses:
    dataframe_mes = dataframes_copia[mes]
    ventas_por_dia_mes = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum().tolist()
    data.append(ventas_por_dia_mes)

# Creo un gráfico de heatmap
fig = go.Figure(data=go.Heatmap(
    z = data,
    x = dias,
    y = meses,  # Invierto el orden de los meses
    customdata = data,
    hovertemplate = '"Día: %{x}" "Mes: %{y}" "Cantidad vendida: %{customdata}<extra></extra>"',
    colorscale = 'RdYlGn',
))

# Configuro los ticks en el eje x para mostrar los números de los días
fig.update_xaxes(
    tickvals = dias,
    title = 'Días',  # Agrego un título al eje x
)

# Le agrego título
fig.update_layout(
    title = 'Cant. Vendida por Día en Diferentes Meses',
)

# Muestro gráfico
fig.show()

In [80]:
# Le aplico la función a cada DF
for mes, df_ventas in dataframes.items():
    dataframes[mes] = extraer_datos(df_ventas,"dia_semana")

# Lista de los días de la semana ordenados en inglés
dias_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Hago un subplots con dos filas y seis columnas
fig = make_subplots(rows=2, cols=6, subplot_titles=meses, shared_xaxes=True, vertical_spacing=0.15)

for mes_index, mes in enumerate(meses):
    df_ventas = dataframes.get(mes, pd.DataFrame())

    if not df_ventas.empty:
        df_ventas['Dia de la Semana'] = pd.Categorical(df_ventas['Dia de la Semana'], categories=dias_semana, ordered=True)

        ventas_por_dia_semana = df_ventas.groupby('Dia de la Semana')['Cantidad Pedida'].sum().reset_index()

        # Uso plotly express para crear el gráfico de barras
        bar_fig = px.bar(
            ventas_por_dia_semana,
            x = 'Dia de la Semana',
            y = 'Cantidad Pedida',
            color = 'Cantidad Pedida',
        )

        # Agrego los valores de 'Cantidad Pedida' al gráfico de barras
        bar_fig.update_traces(text = ventas_por_dia_semana['Cantidad Pedida'], textposition = 'outside')

        # Agrego el gráfico de barras al subplot que corresponda
        fila = mes_index // 6 + 1
        columna = mes_index % 6 + 1
        fig.add_trace(bar_fig.data[0], row=fila, col=columna)

# Ajusto el ancho y el alto del gráfico
fig.update_layout(
    height = 800,
    width = 1700,
)

# Muestro gráfico
fig.show()